In [71]:
import pandas as pd
import scipy.stats as st

In [2]:
import hvplot.pandas
hvplot.extension('plotly')

In [3]:
import os
os.chdir("/home/kantundpeterpan/projects/zoomcamp/zcmlops/week1/train_model")

In [4]:
def read_df(filename: str):
    df = pd.read_parquet(filename)
    # calc duration
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df['duration'] = df.duration.apply(lambda x: x.total_seconds() / 60)
    # feature columns
    categorical = ['PULocationID', 'DOLocationID']
    # numerical = ['trip_distance']
    df[categorical] = df[categorical].astype(str)
    df = df.query("duration > 1 & duration <= 60")
    return df
    

In [134]:
df_train = read_df('./data/green_tripdata_2021-01.parquet')
df_val = read_df('./data/green_tripdata_2021-02.parquet')

In [66]:
[*map(len, (df_train, df_val))]

[73833, 61889]

In [80]:
df_train.duration.hvplot.hist(bins=30, label='train') * \
    df_val.duration.hvplot.hist(bins=30, label = 'Val') \
           .opts(title = 'Distribution of trip duration')

:Overlay
   .Histogram.I  :Histogram   [duration]   (Count)
   .Histogram.II :Histogram   [duration]   (Count)

In [414]:
from functions import fit_and_plot_gamma

In [81]:
fit_and_plot_gamma(df_train.duration) * fit_and_plot_gamma(df_val.duration) 

(np.float64(1.8885994221959685), np.float64(0.9155580841005648), np.float64(8.447047445696032))
(np.float64(1.9088803255927158), np.float64(0.9345386737115762), np.float64(8.865311562466793))


:Overlay
   .Histogram.I  :Histogram   [duration]   (Frequency)
   .Curve.I      :Curve   [x]   (pdf)
   .Histogram.II :Histogram   [duration]   (Frequency)
   .Curve.II     :Curve   [x]   (pdf)

In [87]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import (LinearRegression, Lasso,
    GammaRegressor, TweedieRegressor)

from sklearn.metrics import mean_squared_error, root_mean_squared_error

In [135]:
df_train['PU_DO'] = df_train['PULocationID'] + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + df_val['DOLocationID']


In [136]:
categorical = ['PULocationID', 'DOLocationID']
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(
    train_dicts
)
val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
    

In [137]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [400]:
# lr = GammaRegressor(alpha=0.01, max_iter=250, verbose=3, fit_intercept=True)
lr = LinearRegression()

lr.fit(X_train, y_train)

LinearRegression()

In [403]:
y_pred = lr.predict(X_val)

In [404]:
pd.Series(y_pred).describe()

count    61889.000000
mean        16.440859
std          9.240283
min         -3.149623
25%          8.949331
50%         14.307026
75%         24.170289
max         63.968991
dtype: float64

In [405]:
hvplot.extension("bokeh")

In [407]:
pd.DataFrame.from_dict(
    {
        'true': y_val,
        'pred': y_pred
    }
).hvplot.kde() * \
    pd.DataFrame.from_dict(
    {
        'true': y_val,
        'pred': y_pred
    }
).hvplot.hist(normed=True, bins = 50, alpha = .5) \
    .opts(title = 'Ground truth vs. predicted')

:Overlay
   .NdOverlay.I  :NdOverlay   [Variable]
      :Distribution   [value]   (Density)
   .NdOverlay.II :NdOverlay   [Variable]
      :Histogram   [pred]   (Frequency)

In [408]:
root_mean_squared_error(y_val, y_pred)

7.785649936418828

In [ ]:
import pickle as pkl

In [411]:
with open("../models/lin_reg.bin", 'wb') as f:
    pkl.dump((dv, lr), f)

In [251]:
# https://stats.stackexchange.com/questions/267247/interpret-dispersion-parameter-in-gamma-glm/650450#650450